In [1]:
import sys
import re
from tqdm import tqdm

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *
from calculus_path_mod.terms import num_comparison, nat_numbers, arithm_ops, combinators, pairs, logic

from calculus_path_mod.terms.pseudonym import *

In [6]:
FIB = """
(Y (@f. (@pp. (
    IF (ISZERO (FIRST pp))
        (FIRST (SECOND pp))
        (f (PAIR
            (PRED (FIRST pp))
            (PAIR
                (SECOND (SECOND pp))
                (PLUS (FIRST (SECOND pp)) (SECOND (SECOND pp)))
            )
            )
        )
    )
))
)
"""

In [3]:
LAMBDA_COMMANDS_DICT = {
    # logic
    "TRUE": "logic.true_term()",
    "FALSE": "logic.false_term()",
    "IF": "logic.ite_term()",
    "ITE": "logic.ite_term()",
    "NOT": "logic.not_term()",
    "AND": "logic.and_term()",
    "OR": "logic.or_term()",

    # combinators
    "K": "combinators.k_term()",
    "K_STAR": "combinators.k_star_term()",
    "S": "combinators.s_term()",
    "I": "combinators.i_term()",
    "Y": "combinators.y_term()",
    "Z": "combinators.z_term()",
    "ETTA_V": "combinators.etta_v_term()",

    # nat numbers
    "NUM": "nat_numbers.num_term(",

    # num comparison
    "ISZERO": "num_comparison.iszero_term()",
    "LEQ": "num_comparison.leq_term()",
    "EQ": "num_comparison.eq_term()",
    "LT": "num_comparison.lt_term()",
    "NEQ": "num_comparison.neq_term()",
    "GEQ": "num_comparison.geq_term()",
    "GT": "num_comparison.gt_term()",

    # pairs
    "PAIR": "pairs.pair_term()",
    "FIRST": "pairs.first_term()",
    "SECOND": "pairs.second_term()",

    # arithm ops
    "SUCC": "arithm_ops.succ_term()",
    "SINC": "arithm_ops.sinc_term()",
    "PRED": "arithm_ops.pred_term()",
    "SUBTRACT": "arithm_ops.subtract_term()",
    "MINUS": "arithm_ops.subtract_term()",
    "DIV": "arithm_ops.div_term()",
    "MOD": "arithm_ops.mod_term()",
    "IDIV": "arithm_ops.idiv_term()",
    "PLUS": "arithm_ops.plus_term()",
    "SUM": "arithm_ops.plus_term()",
    "MULT": "arithm_ops.mult_term()",
}


def tokenize_term(lambda_code) -> list:
    brackets_counter = 0
    is_not_tokenized = True

    is_not_space_delimited = True

    lambda_code_tokenized = ""
    for symbol in lambda_code:
        if symbol == "(":
            brackets_counter += 1
        if symbol == ")":
            brackets_counter -= 1
        if brackets_counter == 0:
            is_not_tokenized = True
        if brackets_counter == 1:
            if is_not_tokenized:
                lambda_code_tokenized += "><"
                is_not_tokenized = False
        if brackets_counter == 0:
            if is_not_space_delimited:
                if symbol == " ":
                    lambda_code_tokenized += "><"
                    is_not_space_delimited = False
            else:
                if symbol != " ":
                    is_not_space_delimited = True

        lambda_code_tokenized += symbol

    lambda_code_tokenized = [token.strip() for token in lambda_code_tokenized.split("><")]
    lambda_code_tokenized = [token for token in lambda_code_tokenized if token != ""]

    return lambda_code_tokenized


def process_tokens_to_pt(lambda_code: str, vars_list: list) -> str:
    # is abstraction
    if re.match(r"\s*\(\s*@\s*[a-zA-Z0-9_-].\s*", lambda_code):
        inx_open = 0
        inx_close = -1

        # remove outer brackets
        while lambda_code[inx_open] != "(":
            inx_open += 1
        while lambda_code[inx_close] != ")":
            inx_close -= 1
        lambda_code = lambda_code[inx_open + 1: inx_close]

        inx_open = 0
        while lambda_code[inx_open] != ".":
            inx_open += 1

        var_name = lambda_code[:inx_open]
        lambda_code = lambda_code[inx_open + 1:]

        for var in vars_list:
            if var in var_name:
                var_name = var
                break

        return f"Lambda({var_name}, {process_tokens_to_pt(lambda_code, vars_list)})"
    else:  # it is an app or a single term
        if ("(" in lambda_code) and (")" in lambda_code):  # it is an app
            inx_open = 0
            inx_close = -1

            # remove outer brackets
            while lambda_code[inx_open] != "(":
                inx_open += 1
            while lambda_code[inx_close] != ")":
                inx_close -= 1
            lambda_code = lambda_code[inx_open + 1: inx_close]

            tokens = tokenize_term(lambda_code)
            if len(tokens) == 0:
                raise Exception("Something went wrong")
            elif len(tokens) == 1:
                return process_tokens_to_pt(tokens[0], vars_list)
            elif len(tokens) == 2:
                return f"App({process_tokens_to_pt(tokens[0], vars_list)}, {process_tokens_to_pt(tokens[1], vars_list)})"
            else:
                result_line = "multi_app_term("
                for token in tokens:
                    result_line += str(process_tokens_to_pt(token, vars_list)) + ", "
                result_line += ")"
                return result_line
        else:  # it is a single term
            lambda_code = lambda_code.strip()
            if lambda_code in LAMBDA_COMMANDS_DICT.keys():
                return LAMBDA_COMMANDS_DICT[lambda_code]
            else:
                if ("." in lambda_code) \
                        or ("@" in lambda_code):
                    raise Exception("Not allowed symbol in Lambda term")
                if ("NUM" in lambda_code) and ("[" in lambda_code) and ("]" in lambda_code):
                    try:
                        num = int(lambda_code.strip().split("[")[1][:-1])
                        return LAMBDA_COMMANDS_DICT["NUM"] + str(num) + ")"
                    except:
                        raise Exception("Can't parse number")
                return lambda_code + "_"
            pass


def convert_to_python_code(lambda_code: str) -> str:
    # remove "\n", "\t", and outer spaces symbol
    lambda_code = lambda_code.replace("\n", "").replace("\t", " ").strip()
    if lambda_code == "":
        raise Exception("Can't convert to lambda term an empty value")

    # check brackets
    count_open_brackets = lambda_code.count("(")
    count_close_brackets = lambda_code.count(")")
    if count_open_brackets != count_close_brackets:
        raise Exception(
            f"Wrong count brackets, can't interpreter this cause '(' = {count_open_brackets}, ')' = {count_close_brackets}")

    # remove redundant spaces
    lambda_code = re.sub(r"\s+", " ", lambda_code)

    # find variables & atom terms
    vars_atoms_list = lambda_code.replace("(", " ").replace(")", " ").replace(".", " ").replace("@", " ")
    vars_atoms_list = re.sub(r"\s+", " ", vars_atoms_list)
    vars_atoms_list = vars_atoms_list.split()
    vars_atoms_list = [va_name for va_name in vars_atoms_list if va_name not in LAMBDA_COMMANDS_DICT.keys()]
    vars_atoms_list = list(set(vars_atoms_list))

    result_line = "def gen_term():\n"
    for var_name in vars_atoms_list:
        result_line += "\t" + var_name + " = Var()\n"
        result_line += "\t" + var_name + f"_ = Atom({var_name})\n"

    result_line += "\n\tresult_term = " + process_tokens_to_pt(lambda_code, vars_atoms_list)
    result_line += "\n\treturn result_term"
    return result_line


def conv_to_term(lambda_code: str) -> Term:
    # define a function for generating a term
    # using exec() and call this function to get result
    exec(convert_to_python_code(lambda_code))
    return eval("gen_term()")

In [7]:
strategy_lo = LOStrategy()
strategy_li = LIStrategy()
strategy_ri = RIStrategy()


def test_term(strategy_name, idx):
    strategy = None
    if strategy_name == "LO":
        strategy = strategy_lo
    elif strategy_name == "LI":
        strategy = strategy_li
    elif strategy_name == "RI":
        strategy = strategy_ri
    if not strategy:
        return

    fib_idx = App(conv_to_term(FIB), multi_app_term(pairs.pair_term(),
                                              nat_numbers.num_term(idx),
                                              multi_app_term(pairs.pair_term(), nat_numbers.num_term(0),
                                                             nat_numbers.num_term(1))))
    norm_term, steps = fib_idx.normalize(strategy, is_limited=False)
    print(f"{strategy_name}: {steps} {norm_term.funky_str()}")

In [8]:
test_term("LO", 0)

LO: 30 (λx.(λy.y))


In [9]:
test_term("LO", 1)

LO: 82 (λx.(λy.(x y)))


In [10]:
test_term("LO", 2)

LO: 187 (λx.(λy.(x y)))


In [11]:
test_term("LO", 3)

LO: 348 (λx.(λy.(x (x y))))


In [12]:
test_term("LO", 4)

LO: 597 (λx.(λy.(x (x (x y)))))
